# Laboratorio 2: Clasificación

Minería de Datos - Mayo 2024

**Pregunta 1**

Para realizar la evaluación de distintos clasificadores, vamos a crear la función `run_classifier()`, la cual evalúa un clasificador `clf` recibido como parámetro, un dataset `X,y` (features y target) y un número de tests llamado `num_test`. Esta función almacena y retorna los valores de precision, recall y f1-score en la variable `metrics` además de los resultados de predicción.

En base a lo anterior, incluya las sentencias que entrenen el modelo junto a su correspondiente predicción sobre los datos. **No use cross-validation ni tampoco el parámetro `random_state`.**


In [1]:
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import numpy as np

def run_classifier(model, X, y, num_tests=100):
    metrics = {'precision': [], 'recall': [], 'f1-score': []}

    for _ in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30)

        model = model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        # 0=Malignant, 1=Benign. In sklearn metrics, positive label is by default=1
        metrics['y_pred'] = predictions
        metrics['precision'].append(precision_score(y_test, predictions, pos_label=0))
        metrics['recall'].append(recall_score(y_test, predictions, pos_label=0))
        metrics['f1-score'].append(f1_score(y_test, predictions, pos_label=0))
    return metrics

Luego de completar el código anterior, ejecute el siguiente bloque para comparar distintos clasificadores.
Usaremos un **dataset de cáncer de mamas** para evaluar. La información del dataset se puede encontrar en el siguiente link: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [2]:
bc = load_breast_cancer()

dataset = bc.data
targets = bc.target
results = {}

c0 = ("Base Dummy", DummyClassifier(strategy='stratified'))
c1 = ("Decision Tree", DecisionTreeClassifier(max_depth=5))
c2 = ("Gaussian Naive Bayes", GaussianNB())
c3 = ("KNN", KNeighborsClassifier(n_neighbors=10))
c4 = ("Support Vector Machines", SVC())

models = [c0, c1, c2, c3, c4]

for name, model in models:
    metrics = run_classifier(model, dataset, targets)
    results[name] = metrics

    print("----------------")
    print("Resultados para clasificador: ", name)
    print("Precision promedio:", np.array(metrics['precision']).mean())
    print("Recall promedio:", np.array(metrics['recall']).mean())
    print("F1-score promedio:", np.array(metrics['f1-score']).mean())
    print("----------------\n\n")

----------------
Resultados para clasificador:  Base Dummy
Precision promedio: 0.38131012935147723
Recall promedio: 0.3706435971048224
F1-score promedio: 0.37380123958298866
----------------


----------------
Resultados para clasificador:  Decision Tree
Precision promedio: 0.9164341659428733
Recall promedio: 0.9020819665323709
F1-score promedio: 0.9081408426656133
----------------


----------------
Resultados para clasificador:  Gaussian Naive Bayes
Precision promedio: 0.946170615225485
Recall promedio: 0.8920917343506893
F1-score promedio: 0.9177258061278757
----------------


----------------
Resultados para clasificador:  KNN
Precision promedio: 0.9373228884208353
Recall promedio: 0.8831124923365306
F1-score promedio: 0.9087424546435764
----------------


----------------
Resultados para clasificador:  Support Vector Machines
Precision promedio: 0.9634421587106455
Recall promedio: 0.793587498011406
F1-score promedio: 0.8689146451717339
----------------




**Pregunta 2**

Analizando los resultados obtenidos de cada clasificador, y basándose en las métricas calculadas. ¿Cuál es el mejor clasificador? ¿Qué métricas observó para tomar esa decisión y por qué? considerando el problema que aborda. Fundamente su respuesta.

(Considere *malignant* como clase positiva, y *benign* como clase negativa.)

**Repuesta:** Considerando el contexto de diagnóstico de cáncer de mamas, la métrica que mejor permite optimizar el modelo es Recall. Por lo tanto el modelo Decision Tree es el mejor clasificador.




# Seleccionar hiperparámetros

**Dataset:** En esta y la siguiente parte del laboratorio utilizaremos el dataset **"ML Classification: Predicting 5-Year Career Longevity for NBA Rookies"** de data.world (https://data.world/ssaudz/ml-classification-predicting-5-year-career-longevity-for-nb). Este dataset contiene estadísticas de los novatos en la NBA y busca predecir si un jugador podrá durar 5 años en la liga. La columna objetivo es *TARGET_5Yrs*. Esta es una versión preprocesada del dataset original (después de eliminar registros duplicados por nombre del jugador, anonimizar los datos, y eliminar los registros con valores nulos).

In [3]:
import pandas as pd
nba_rookies = pd.read_csv('https://raw.githubusercontent.com/cinthiasanchez/data-mining/main/NBA_career_longevity.csv')
nba_rookies.shape

(1284, 20)

In [4]:
nba_rookies.head(2)

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0
1,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0


In [5]:
# Separando atributos predictores (X) del atributo objetivo (y)
dataset = nba_rookies.iloc[:,:-1].values
targets = nba_rookies['TARGET_5Yrs'].values

print(nba_rookies.iloc[:,:-1])
print(dataset)
print(targets)

# dividiendo los datos de entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(dataset, targets, test_size=.30,
                                                    random_state=15, stratify=targets)

      GP   MIN  PTS  FGM  FGA   FG%  3P Made  3PA   3P%  FTM  FTA   FT%  OREB  \
0     36  27.4  7.4  2.6  7.6  34.7      0.5  2.1  25.0  1.6  2.3  69.9   0.7   
1     35  26.9  7.2  2.0  6.7  29.6      0.7  2.8  23.5  2.6  3.4  76.5   0.5   
2     74  15.3  5.2  2.0  4.7  42.2      0.4  1.7  24.4  0.9  1.3  67.0   0.5   
3     58  11.6  5.7  2.3  5.5  42.6      0.1  0.5  22.6  0.9  1.3  68.9   1.0   
4     48  11.5  4.5  1.6  3.0  52.4      0.0  0.1   0.0  1.3  1.9  67.4   1.0   
...   ..   ...  ...  ...  ...   ...      ...  ...   ...  ...  ...   ...   ...   
1279  80  15.8  4.3  1.6  3.6  43.3      0.0  0.2  14.3  1.2  1.5  79.2   0.4   
1280  68  12.6  3.9  1.5  4.1  35.8      0.1  0.7  16.7  0.8  1.0  79.4   0.4   
1281  43  12.1  5.4  2.2  3.9  55.0      0.0  0.0   0.0  1.0  1.6  64.3   1.5   
1282  52  12.0  4.5  1.7  3.8  43.9      0.0  0.2  10.0  1.2  1.8  62.5   0.2   
1283  47  11.7  4.4  1.6  4.4  36.9      0.4  1.3  33.3  0.7  1.0  67.3   0.2   

      DREB  REB  AST  STL  

In [6]:
print(nba_rookies[["MIN", "PTS"]])
print(nba_rookies[["MIN", "PTS"]].values)
print(nba_rookies[["MIN", "PTS"]].values.shape)

       MIN  PTS
0     27.4  7.4
1     26.9  7.2
2     15.3  5.2
3     11.6  5.7
4     11.5  4.5
...    ...  ...
1279  15.8  4.3
1280  12.6  3.9
1281  12.1  5.4
1282  12.0  4.5
1283  11.7  4.4

[1284 rows x 2 columns]
[[27.4  7.4]
 [26.9  7.2]
 [15.3  5.2]
 ...
 [12.1  5.4]
 [12.   4.5]
 [11.7  4.4]]
(1284, 2)


## GridSearchCV

Una alternativa para seleccionar hiperparámetros es GridSearchCV, la cual considera exhaustivamente todas las combinaciones de parámetros. GridSearchCV recibe un `modelo: Model`, `param_dict: dict`, `scoring: str | list | dict` y un objeto `k_folds: KFold`. Ver la documentación en: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

knn_param_dict = {
  'n_neighbors': [1, 3, 5, 10],
  'weights': ['uniform', 'distance']
}

knn_cv = GridSearchCV(KNeighborsClassifier(),
                      param_grid=knn_param_dict,
                      cv=5)

knn_cv.fit(X_train, y_train)

print("Mejor combinación de parámetros (sin scoring='f1'):")
print(knn_cv.best_params_)

print("\nClassification Report:")
y_pred = knn_cv.predict(X_test)
print(classification_report(y_test, y_pred))

Mejor combinación de parámetros (sin scoring='f1'):
{'n_neighbors': 10, 'weights': 'distance'}

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.54      0.58       145
           1       0.75      0.82      0.78       241

    accuracy                           0.71       386
   macro avg       0.69      0.68      0.68       386
weighted avg       0.71      0.71      0.71       386



In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import numpy as np

knn_param_dict = {
  'n_neighbors': np.arange(5, 16),
  'weights': ['uniform', 'distance']
}

knn_cv = GridSearchCV(KNeighborsClassifier(),
                      param_grid=knn_param_dict,
                      cv=5)

knn_cv.fit(X_train, y_train)

print("Mejor combinación de parámetros (sin scoring='f1'):")
print(knn_cv.best_params_)

print("\nClassification Report:")
y_pred = knn_cv.predict(X_test)
print(classification_report(y_test, y_pred))

Mejor combinación de parámetros (sin scoring='f1'):
{'n_neighbors': 11, 'weights': 'distance'}

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.59      0.63       145
           1       0.77      0.83      0.80       241

    accuracy                           0.74       386
   macro avg       0.73      0.71      0.72       386
weighted avg       0.74      0.74      0.74       386



In [9]:
np.arange(5, 16)

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [10]:
nba_rookies['TARGET_5Yrs'].value_counts()

TARGET_5Yrs
1    802
0    482
Name: count, dtype: int64

**Pregunta 3**

Al ejecutar el bloque de código anterior, ¿Qué puede decir de los resultados, con cuáles parámetros los obtuvo? ¿Cuál considera que es la principal ventaja de aplicar GridSearchCV? ¿Considera que es necesario seguir explorando los parámetros?

**Repuesta:**
Los hiperparámetros optimizados para KNN (tuned) fueron `n_neighbors` y `weights`.

Los resultados indican que con `"n_neighbors"=10` y `"weights": "distance"` se obtienen los mejores resultados.

La principal ventaja de `GridSearchCV` es que identifica el valor de hiperparámetros que permite maximizar el resultado respecto a las clases target.

Explorando mejor los valores cerca del `10` obtenido al principio, se obtiene un mejor hiperparámetro `n_neighbors=11`.

# Trabajar con clases desbalanceadas

Al explorar el dataset anterior, se nota un desbalance importante (38%-62%). Para mejorar el rendimiento de un clasificador sobre clases desbalanceadas existen varias técnicas. En esta parte, veremos cómo tratar con este problema usando (sub/over) sampling de las clases.

Note el desbalance de las clases ejecutando el siguiente código:

In [11]:
print("Distribucion de clases original:")
print(nba_rookies['TARGET_5Yrs'].value_counts())
print(nba_rookies['TARGET_5Yrs'].value_counts() / len(nba_rookies) * 100)

Distribucion de clases original:
TARGET_5Yrs
1    802
0    482
Name: count, dtype: int64
TARGET_5Yrs
1    62.461059
0    37.538941
Name: count, dtype: float64


Antes de hacer algo para tratar el desbalance entre las clases primero debemos dividir en train-test. Como ya hicimos la partición de train y test, vamos a explorarla a continuación.

In [12]:
# Dividimos igual que arriba.
# Para facilitar el balance manual, X contendrá el dataset completo, pero luego eliminaremos de éste el atributo objetivo.

X_train_imb, X_test_imb, y_train_imb, y_test_imb = train_test_split(
    nba_rookies,
    nba_rookies['TARGET_5Yrs'],
    test_size=.30,
    random_state=15,
    stratify=nba_rookies['TARGET_5Yrs'])

print("Cantidad de instancias por clase en el y_train_imb:")
print("Clase 1: " + str((y_train_imb == 1).sum()))
print("Clase 0: " + str((y_train_imb == 0).sum()))

Cantidad de instancias por clase en el y_train_imb:
Clase 1: 561
Clase 0: 337



Aplicaremos **oversampling** y **subsampling** al train para que queden balanceados. Ejecute el siguiente código y note ahora que las clases están balanceadas.

In [13]:
print("Distribución de clases usando (over/sub) sampling: \n")
X_train_imb = X_train_imb.reset_index(drop=True) # resets the index column with a new sequence

### Oversampling sobre la clase 0
target_5yrs_no = X_train_imb[X_train_imb['TARGET_5Yrs'] == 0]
random_index_subset_0 = np.random.choice(target_5yrs_no.index, size=224)
print("random_index_subset_0:", len(random_index_subset_0))

# print(len(X_train_imb.index), " + ", len(random_index_subset_0))
oversampled_data = pd.concat([X_train_imb, X_train_imb.iloc[random_index_subset_0]])
print("Data oversampled on class '0'")
print(oversampled_data['TARGET_5Yrs'].value_counts(), "\n")

### Subsampling sobre la clase 1
target_5yrs_yes = X_train_imb[X_train_imb['TARGET_5Yrs'] == 1]
random_index_subset_1 = np.random.choice(target_5yrs_yes.index, size=224, replace=False)
print("random_index_subset_1:", len(random_index_subset_1))

# print(len(X_train_imb.index), " + ", len(random_index_subset_1))
undersampled_data = X_train_imb.drop(X_train_imb.iloc[random_index_subset_1].index)
print("Data undersampled on class '1'")
print(undersampled_data['TARGET_5Yrs'].value_counts())

Distribución de clases usando (over/sub) sampling: 

random_index_subset_0: 224
Data oversampled on class '0'
TARGET_5Yrs
1    561
0    561
Name: count, dtype: int64 

random_index_subset_1: 224
Data undersampled on class '1'
TARGET_5Yrs
0    337
1    337
Name: count, dtype: int64


**Nota:** *Librerías como `imbalanced-learn` son muy útiles para balancear los datos.*

**Pregunta 4**

¿Por qué aplicar subsampling/oversampling de las clases sobre el conjunto de entrenamiento en lugar de aplicarlo sobre el dataset completo? Argumente su respuesta.

**Respuesta:**

Porque queremos solo afectar la data con la que entrenamos el modelo, no queremos sobrerepresentar las clases minoritarias en la data de prueba.

In [14]:
X_test_imb

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
236,58,13.1,2.7,1.1,2.9,37.4,0.1,0.4,24.0,0.4,0.6,71.9,0.3,0.7,1.0,3.1,0.3,0.1,1.0,0
302,80,20.0,8.5,3.2,5.7,55.8,0.0,0.0,0.0,2.1,3.2,67.3,1.2,2.7,3.9,1.1,0.9,0.5,1.9,1
1117,71,15.5,7.6,2.5,5.9,41.7,0.7,1.9,37.5,1.9,2.3,81.9,0.7,1.2,1.9,1.3,0.7,0.3,1.6,1
735,59,24.4,8.8,3.3,8.7,38.1,1.3,4.0,33.2,0.8,1.0,82.5,0.3,2.7,3.0,1.3,0.4,0.1,1.1,1
130,82,19.1,9.1,3.6,9.2,39.7,0.1,0.5,27.3,1.7,3.1,55.0,1.3,2.4,3.8,0.9,0.7,0.7,1.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,82,31.5,11.4,5.2,11.3,46.1,0.3,1.0,27.7,0.8,1.3,58.9,0.7,2.0,2.6,6.8,1.6,0.3,1.7,1
696,82,32.5,15.3,5.3,13.4,39.8,0.4,1.6,27.1,4.3,5.2,81.5,2.2,2.7,4.9,5.3,1.3,0.2,3.3,1
62,79,37.7,18.0,6.9,16.5,42.2,0.0,0.1,27.3,4.1,5.5,73.6,2.4,6.4,8.8,4.0,1.4,1.1,2.9,1
681,63,11.2,3.9,1.7,3.4,49.5,0.0,0.0,0.0,0.5,0.8,64.6,0.9,1.7,2.5,0.2,0.4,0.3,0.2,1


In [15]:
## ejecutar este código para preparar los datos
from sklearn.metrics import classification_report

# Preparando los data frames para ser compatibles con sklearn

# datos test (mismo para todos los conjuntos de entrenamiento)
X_test, y_test = X_test_imb[X_train_imb.columns[:-1]], X_test_imb['TARGET_5Yrs']

# datos entrenamiento "originales"
X_train_orig = X_train_imb[X_train_imb.columns[:-1]]
y_train_orig = X_train_imb[X_train_imb.columns[-1]]

# datos entrenamiento oversampleados
X_train_over = oversampled_data[X_train_imb.columns[:-1]]
y_train_over = oversampled_data[X_train_imb.columns[-1]]

# datos entrenamiento undersampleados
X_train_under = undersampled_data[X_train_imb.columns[:-1]]
y_train_under = undersampled_data[X_train_imb.columns[-1]]


**Pregunta 5**

Complete el código necesario para entrenar un clasificador KNeighbors en cada uno de los tres casos (**original**, con **oversampling** y con **subsampling**) y luego compare los resultados sobre el conjunto de test (este es el mismo para los tres casos) obtenido con train_test_split sobre los datos originales. Muestre Precision, Recall y F1-score.

Emplee como datos de entrada lo del bloque anterior.

In [16]:
from sklearn.neighbors import KNeighborsClassifier

## Pasos:
##  - instanciar el clasificador con KNeighborsClassifier()
##  - entrenar con fit()
##  - hacer las predicciones
##  - mostrar precision, recall y f1-score con classification report.

print("==ORIGINAL==")
knn = KNeighborsClassifier()
knn.fit(X_train_orig, y_train_orig)
y_pred_original = knn.predict(X_test)
print(classification_report(y_test, y_pred_original))

print("==n_neighbors OPTIMIZED==")
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train_orig, y_train_orig)
y_pred_original = knn.predict(X_test)
print(classification_report(y_test, y_pred_original))

print("==OVERSAMPLED==")
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train_over, y_train_over)
y_pred_over = knn.predict(X_test)
print(classification_report(y_test, y_pred_over))

print("==UNDERSAMPLED==")
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train_under, y_train_under)
y_pred_under = knn.predict(X_test)
print(classification_report(y_test, y_pred_under))

==ORIGINAL==
              precision    recall  f1-score   support

           0       0.60      0.54      0.57       145
           1       0.74      0.78      0.76       241

    accuracy                           0.69       386
   macro avg       0.67      0.66      0.66       386
weighted avg       0.69      0.69      0.69       386

==n_neighbors OPTIMIZED==
              precision    recall  f1-score   support

           0       0.67      0.59      0.62       145
           1       0.77      0.83      0.80       241

    accuracy                           0.74       386
   macro avg       0.72      0.71      0.71       386
weighted avg       0.73      0.74      0.73       386

==OVERSAMPLED==
              precision    recall  f1-score   support

           0       0.55      0.68      0.60       145
           1       0.77      0.66      0.71       241

    accuracy                           0.67       386
   macro avg       0.66      0.67      0.66       386
weighted avg       

**Pregunta 6**

- Observe los resultados obtenidos por clase con cada conjunto de entrenamiento, ¿se puede observar alguna diferencia importante?
- Indique una desventaja de usar oversampling y una desventaja de usar subsampling en clasificación.

**Respuesta**



**Pregunta 7**

Compare los resultados del caso **ORIGINAL** (donde el clasificador usa los parámetros por defecto KNeighborsClassifier()) versus el resultado de la pregunta 3 donde usa los mejores parámetros con GridSearchCV. ¿Qué opina de los resultados?

**Respuesta**
